In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

In [ ]:
pc =Pinecone(api_key=os.getenv("PINECONE_DATABASE_SECRET_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,  
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
import json
data =json.load(open('reviews.json'))
data['reviews']

In [ ]:
from os import getenv


processed_data = []
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("OPENROUTER_API_KEY"),
)

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["proffessor"],
        "metadata": {
        "review" :review["review"],
        "subject":review["subject"],
        "stars" : review["stars"]
        }
    })

In [ ]:
processed_data[0]

In [ ]:

index= pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)